<a href="https://colab.research.google.com/github/sunilypatil/starter-slapp-app/blob/master/rotation_nvr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import numpy as np
import random
from PIL import Image, ImageDraw, ImageFont
import os
import math
from PIL import ImageFont
from math import sin, cos, radians

font = ImageFont.load_default()

from PIL import ImageOps

QUESTION_SIZE = 150

def generate_random_shape():
    shapes = ['circle', 'semicircle', 'oval', 'triangle', 'square', 'rectangle', 
              'parallelogram', 'rhombus', 'trapezoid', 'kite', 'pentagon', 'hexagon', 
              'heptagon', 'octagon']
    shape = random.choice(shapes)
    return shape

def draw_shape(draw, shape, bounding_box, fill='black'):
    x1, y1, x2, y2 = bounding_box
    center = ((x1 + x2) // 2, (y1 + y2) // 2)
    radius = min(abs(x2 - x1), abs(y2 - y1)) // 2

    if shape == 'circle':
        draw.ellipse(bounding_box, fill=fill)
    elif shape == 'semicircle':
        draw.pieslice(bounding_box, start=0, end=180, fill=fill)
    elif shape == 'oval':
        draw.ellipse(bounding_box, fill=fill)
    elif shape == 'triangle':
        triangle_points = [(x1, y2), (x2, y2), center]
        draw.polygon(triangle_points, fill=fill)
    elif shape == 'square':
        draw.rectangle(bounding_box, fill=fill)
    elif shape == 'rectangle':
        draw.rectangle(bounding_box, fill=fill)
    elif shape == 'parallelogram':
        points = [(x1, y2), (x1 + (x2-x1)//3, y1), (x2, y1), (x2 - (x2-x1)//3, y2)]
        draw.polygon(points, fill=fill)
    elif shape == 'rhombus':
        points = [(center[0], y1), (x2, center[1]), (center[0], y2), (x1, center[1])]
        draw.polygon(points, fill=fill)
    elif shape == 'trapezoid':
      # For trapezoid
        points = [(x1 + (x2-x1)//4, y2), (x1 + 3*(x2-x1)//4, y2), (x2, y1), (x1, y1)]
        draw.polygon(points, fill=fill)
    elif shape == 'kite':
        points = [(center[0], y1), (x2, center[1]), (center[0], y2), (x1, center[1])]
        draw.polygon(points, fill=fill)
    else:
        # For regular polygons
        num_sides = {'pentagon': 5, 'hexagon': 6, 'heptagon': 7, 'octagon': 8}[shape]
        points = []
        for i in range(num_sides):
            angle = radians(float(i) / num_sides * 360.0)
            x = center[0] + radius * cos(angle)
            y = center[1] + radius * sin(angle)
            points.append((x, y))
        draw.polygon(points, fill=fill)

def draw_square(draw, pos, size):
    x, y = pos
    draw.rectangle([x - size // 2, y - size // 2, x + size // 2, y + size // 2], outline="black", width=2)

def draw_circle(draw, pos, size):
    x, y = pos
    draw.ellipse([x - size // 2, y - size // 2, x + size // 2, y + size // 2], outline="black", width=2)

def draw_triangle(draw, pos, size):
    x, y = pos
    points = [(x, y - size // 2), (x + size // 2, y + size // 2), (x - size // 2, y + size // 2), (x, y - size // 2)]
    for i in range(3):
        draw.line([points[i], points[i + 1]], fill="black", width=2)


def draw_shape_with_points(draw, points, size):
    for i, point in enumerate(points):
        # Draw arcs between points
        angle = math.atan2(point[1] - points[i - 1][1], point[0] - points[i - 1][0])
        start_angle = math.degrees(angle)
        end_angle = math.degrees(angle + math.pi / len(points))

        bounding_box = [point[0] - size / 2, point[1] - size / 2, point[0] + size / 2, point[1] + size / 2]
        draw.arc(bounding_box, start_angle, end_angle, fill=(0, 0, 0), width=2)

    # Connect the last point to the first point
    draw.line([points[-1], points[0]], fill=(0, 0, 0), width=2)

def generate_complex_shape(draw, center, size, num_sides, angle_offset, arc_direction):
    # Create a complex shape using the provided attributes
    points = []
    for i in range(num_sides):
        angle = math.radians(i * 360 / num_sides + angle_offset)
        x = center[0] + size * math.cos(angle)
        y = center[1] + size * math.sin(angle)
        points.append((x, y))

        # Draw arcs between points
        start_angle = math.degrees(angle)
        end_angle = math.degrees(angle + math.pi / num_sides)
        if arc_direction:
            start_angle, end_angle = end_angle, start_angle

        bounding_box = [points[-1][0] - size / 2, points[-1][1] - size / 2, points[-1][0] + size / 2, points[-1][1] + size / 2]
        draw.arc(bounding_box, start_angle, end_angle, fill=(0, 0, 0), width=2)

    # Connect the last point to the first point
    draw.line([points[-1], points[0]], fill=(0, 0, 0), width=2)

    return points


def draw_arrow(draw, center, direction):
    if direction == "clockwise":
        points = [(center[0], center[1] - 10), (center[0] + 10, center[1]), (center[0], center[1] + 10)]
    else:
        points = [(center[0], center[1] - 10), (center[0] - 10, center[1]), (center[0], center[1] + 10)]
    draw.polygon(points, fill=(0, 0, 0))

def create_rotation_options(question, direction, degrees):
    answer_options = []
    rotations = [0, 90, 180, 270]
    random.shuffle(rotations)

    for rotation in rotations:
        option = question.copy()

        if direction == "clockwise":
            rotated_option = option.rotate(-rotation, Image.NEAREST, expand=1)
        else:
            rotated_option = option.rotate(rotation, Image.NEAREST, expand=1)

        rotated_option = rotated_option.resize((QUESTION_SIZE, QUESTION_SIZE), Image.LANCZOS)
        answer_options.append(rotated_option)

    if direction == "clockwise":
        correct_answer = rotations.index(degrees) + 1
    else:
        correct_answer = rotations.index(360 - degrees) + 1

    return answer_options, correct_answer


def generate_rotation_question(complexity='easy'):
    question = Image.new('RGBA', (QUESTION_SIZE, QUESTION_SIZE), 'white')
    draw = ImageDraw.Draw(question)

    center = (QUESTION_SIZE // 2, QUESTION_SIZE // 2)
    size = 50
    
    # Map complexity to a numerical value
    complexity_map = {'easy': 1, 'medium': 2, 'hard': 3}
    complexity_value = complexity_map[complexity]

    # Get a random shape
    main_shape = generate_random_shape()

    # Choose the rotation direction
    direction = random.choice(["clockwise", "counterclockwise"])
    degrees = random.choice([90, 180, 270])

    # Draw the main shape
    bounding_box = [center[0] - size, center[1] - size, center[0] + size, center[1] + size]
    draw_shape(draw, main_shape, bounding_box)

    # Add additional shapes or arrows based on complexity
    for _ in range(complexity_value):
        # Add additional shape
        additional_shape = generate_random_shape()
        bounding_box = [random.randint(0, QUESTION_SIZE), random.randint(0, QUESTION_SIZE),
                        random.randint(0, QUESTION_SIZE), random.randint(0, QUESTION_SIZE)]
        draw_shape(draw, additional_shape, bounding_box, fill='black')
        
        # Draw a random arrow
        arrow_center = (random.randint(0, QUESTION_SIZE), random.randint(0, QUESTION_SIZE))
        arrow_direction = random.choice(["clockwise", "counterclockwise"])
        draw_arrow(draw, arrow_center, arrow_direction)

    # Create answer options
    answer_options, correct_answer = create_rotation_options(question, direction, degrees)

    question_text = f"What picture do you get when you rotate the shape by {degrees} degrees {direction}?"
    explanation = f"The correct answer is option {correct_answer} because the shape is rotated by {degrees} degrees {direction}."

    return question, answer_options, correct_answer, question_text, explanation, complexity



def generate_random_shape_attributes():
    shape_attributes = {
        'num_sides': random.randint(3, 7),
        'angle_offset': random.randint(0, 360),
        'arc_direction': random.choice([True, False]),
    }
    return shape_attributes


NUM_SIDES = [3, 4, 5, 6, 7, 8]
ANGLE_OFFSETS = [0, 45, 90, 135, 180, 225, 270, 315]
ARC_DIRECTIONS = ['clockwise', 'counterclockwise']

def draw_complex_pattern(draw, center, radius, num_elements):
    for _ in range(num_elements):
        # Choose a random position for the shape
        x = random.randint(radius, QUESTION_SIZE - radius)
        y = random.randint(radius, QUESTION_SIZE - radius)

        # Choose a random number of sides for the shape
        num_sides = random.choice([3, 4, 5, 6])

        # Choose a random direction for the arc
        arc_direction = random.choice([-1, 1])

        # Draw the shape
        generate_complex_shape(draw, (x, y), radius, num_sides, 0, arc_direction)


def draw_advanced_pattern(image, num_shapes):
    draw = ImageDraw.Draw(image)
    shapes = ['circle', 'square', 'triangle', 'semicircle', 'oval', 'rectangle', 
              'parallelogram', 'rhombus', 'trapezoid', 'kite', 'pentagon', 
              'hexagon', 'heptagon', 'octagon']

    for _ in range(num_shapes):
        # Choose a random shape
        shape = random.choice(shapes)
        
        # Choose random parameters for the shape
        bounding_box = (random.randrange(0, QUESTION_SIZE), random.randrange(0, QUESTION_SIZE),
                        random.randrange(0, QUESTION_SIZE), random.randrange(0, QUESTION_SIZE))
        
        # Draw the shape
        draw_shape(draw, shape, bounding_box, fill='black')

    return image


def add_border(image, border_width=2, border_color=(0, 0, 0)):
    width, height = image.size
    bordered_image = Image.new('RGBA', (width + 2 * border_width, height + 2 * border_width), border_color)
    bordered_image.paste(image, (border_width, border_width))
    return bordered_image

QUESTION_TYPES = {
    'rotation': generate_rotation_question
}



In [20]:
def generate_question_set(topic, num_questions, complexity):
    question_set = []
    for _ in range(num_questions):
        if topic == 'rotation':
            question, answer_options, correct_answer, question_text, explanation, complexity = generate_rotation_question(complexity)
            question_info = {
                'question': question,
                'answer_options': answer_options,
                'correct_answer': correct_answer,
                'question_text': question_text,
                'explanation': explanation,
                'complexity': complexity
            }
            question_set.append(question_info)
        # You can add other topics in similar way
    return question_set






In [21]:
def save_images(question_set, output_folder):

    if os.path.exists(output_folder):
        shutil.rmtree(output_folder)

    os.makedirs(output_folder)

    with open(os.path.join(output_folder, 'explanations.txt'), 'w') as f:
        for idx, question in enumerate(question_set, start=1):
            if isinstance(question['question'], list):
                for img_idx, img in enumerate(question['question'], start=1):
                    img.save(os.path.join(output_folder, f'question_{idx}_part_{img_idx}.png'))
            else:
                question['question'].save(os.path.join(output_folder, f'question_{idx}.png'))

            for option_idx, option in enumerate(question['answer_options'], start=1):
                option.save(os.path.join(output_folder, f'question_{idx}_option_{option_idx}.png'))
            
            f.write(f"Question {idx}:\n")
            f.write(question['explanation'] + '\n\n')




In [22]:
import shutil
from google.colab import files

def create_zip_and_download(question_set, output_folder='output', zip_name='questions.zip'):
    # Save images to output folder
    save_images(question_set, output_folder)

   # Save question data as a text file
    with open(os.path.join(output_folder, 'question_data.txt'), 'w') as f:
        for idx, question in enumerate(question_set, start=1):
            f.write(f"Question {idx}: {question['question_text']}\n")
            for i, option in enumerate(question['answer_options'], start=1):
                f.write(f"Option {i}: {option}\n")
            f.write(f"Correct Answer: Option {question['correct_answer']}, Difficulty: {question['complexity']}\n")
            f.write(f"Explanation: {question['explanation']}\n\n")

    # Create a zip file containing the output folder
    shutil.make_archive(zip_name[:-4], 'zip', output_folder)

    # Download the zip file
    files.download(zip_name)




In [23]:
if __name__ == '__main__':
    topic = input("Enter the topic: ").lower()
    num_questions = int(input("Enter the number of questions: "))
    complexity = input("Enter the complexity level (easy, medium, hard): ")

    if topic not in QUESTION_TYPES:
        print("Invalid topic. Available topics are:", ', '.join(QUESTION_TYPES.keys()))
        exit()
        
    question_set = generate_question_set(topic, num_questions, complexity)
    output_folder = "question_set"  # You can change this to any folder name you prefer
    save_images(question_set, output_folder)
    create_zip_and_download(question_set, output_folder)
    print("Question set generated successfully!")


Enter the topic: rotation
Enter the number of questions: 23
Enter the complexity level (easy, medium, hard): easy


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Question set generated successfully!
